<a href="https://colab.research.google.com/github/kok554/computervision/blob/main/VGG%EA%B3%B5%EB%B6%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [2]:
# NVIDIA의 CUDA 컴파일러 드라이버의 버전을 확인하는 명령어
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
# 필수 Package import
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models

In [4]:
# 이미지를 출력하는 함수
def display_images(image_paths, title, max_images=4):
  """ 지정된 이미지 경로의 이미지를 최대 4개까지 화면에 출력하는 함수 """
  plt.figure(figsize=(12, 3))
  for i, image_path in enumerate(image_paths[:max_images]):
    # 이미지 파일을 읽어옴
    img = plt.imread(image_path)

    # 서브플롯을 생성하여 이미지를 배치
    plt.subplot(1, max_images, i + 1) # 1 행 max_images열의 서브플롯 설정
    plt.imshow(img)  # 이미지 서브플롯에 표시
    plt.title(title)
    plt.axis('off')

  plt.show()

In [4]:
# 이미지 카테고리 목록 정의
categories = ['Train crack', 'Train normal', 'Val crack', 'Val normal', 'Test crack', 'Test normal']

# 각 카테고리에 대해 반복
for category in categories:
    # 해당 카테고리에 맞는 이미지 경로를 glob을 사용하여 가져옴
    # 경로는 카테고리 이름을 소문자로 변환하고 공백을 '/'로 대체하여 생성
    image_paths = glob.glob(f'/content/drive/MyDrive/Colab Notebooks/ComputerVision/3/{category.lower().replace(" ","/")}/*')

    # 이미지 경로를 사용하여 이미지를 표시하는 함수 호출
    display_images(image_paths, category)

    # 현재 카테고리의 총 이미지 수를 출력
    print(f"{category} 총 이미지 수: {len(image_paths)}")

# 전체 이미지 수를 시각화하기 위한 막대 그래프 생성
plt.figure(figsize=(10, 6))
# 각 카테고리의 이미지 수를 계산하여 막대 그래프의 높이로 사용
plt.bar(categories, [len(glob.glob(f'/content/drive/MyDrive/Colab Notebooks/ComputerVision/3/{category.lower().replace(" ","/")}/*')) for category in categories])
# 그래프 제목 및 축 레이블 설정
plt.title('Number of Images per Category')
plt.xlabel('Category')
plt.ylabel('Number of Images')
# x축 레이블을 45도 회전하여 가독성 향상
plt.xticks(rotation=45)
# 그래프 표시
plt.show()

In [5]:
# 데이터 전처리 정의
transform = transforms.Compose([
    # 이미지를 224 x 224 크기로 조정
    transforms.Resize((224, 224)),

    # 이미지를 텐서로 변환
    transforms.ToTensor(),

    # 이미지 정규화: 각 채널의 평균을 0.5로, 표준편차를 0.5로 설정
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 데이터셋 로드 및 데이터 로더 생성
train_dataset = ImageFolder('/content/drive/MyDrive/Colab Notebooks/ComputerVision/3/train', transform=transform)
val_dataset = ImageFolder('/content/drive/MyDrive/Colab Notebooks/ComputerVision/3/val', transform=transform)


In [6]:
from torch.utils.data import SubsetRandomSampler
import numpy as np

# 데이터셋의 인덱스를 생성
num_of_train = 34550
num_of_val = 330

# 훈련 데이터와 검증 데이터의 인덱스 리스트 생성
train_indices = list(range(num_of_train))
val_indices = list(range(num_of_val))

# 200개의 무작위 샘플 선택 (학습 데이터 수가 많아 실습을 진행하기 제한이 있어 설정)
np.random.shuffle(train_indices) # 훈련 데이터 인덱스를 무작위로 섞음
train_subset_indices = train_indices[:200] # 섞인 인덱스 중 처음 200개

np.random.shuffle(val_indices) # 검증 데이터 인덱스를 무작위로 섞음
val_subset_indices = val_indices[:200] # 섞인 인덱스 중 처음 200개

# 사용자 정의 Sampler 생성
train_sampler = SubsetRandomSampler(train_subset_indices)
val_sampler = SubsetRandomSampler(val_subset_indices)

# DataLoader에 Sampler 지정
train_loader = DataLoader(dataset=train_dataset, batch_size =5, sampler=train_sampler)
val_loader = DataLoader(dataset=val_dataset, batch_size =5, sampler=val_sampler)

In [7]:
# VGG 모델 로드 및 네트워크 구조 확인
net = models.vgg19(pretrained=True)
net

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 92.4MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [8]:
# 모델의 모든 파라미터를 고정
for param in net.parameters():
  param.requires_grad = False # 각 파라미터의 requires_grad 속성을 False로 설정

In [9]:
# classifier의 마지막 레이어를 Binary Classification Task에 맞게 교체하고, 이 레이어의 파라미터는 학습 가능하도록 설정
net.classifier[6] = nn.Linear(4096, 2) # 마지막 레이어를 2개의 출력 노드를 가진 Linear 레이어로 교체
net.classifier[6].requires_grad = True # 새로운 레이어의 파라미터는 학습 가능하도록 설정

In [10]:
# 손실함수
creiterion = nn.CrossEntropyLoss() # 크로스 엔트로피 손실 함수 정의

In [11]:
import torch.optim as optim # PyTorch의 최적화 모듈 임포트

def train_model(optimizer_name, net, train_loader, val_loader, criterion, num_epochs=20):
  # optimizer 설정
  if optimizer_name == 'SGD':
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # SGD 최적화기 설정
  elif optimizer_name == 'Adam':
    optimizer = optim.Adam(net.parameters(), lr=0.001) # Adam 최적화기 설정
  elif optimizer_name == 'RAdam':
    optimizer = optim.RAdam(net.parameters(), lr=0.001) # RAdam 최적화기 설정
  else:
    raise ValueError("Invalid optimizer name. Choose 'SGD', 'Adam', or 'RAdam'.")


  # 학습/검증 손실과 검증 정확도를 저장할 리스트
  train_losses = [] # 학습 손실을 저장할 리스트
  val_losses = []  # 검증 손실을 저장할 리스트
  val_accuracies = [] # 검증 정확도를 저장할 리스트

  for epoch in range(num_epochs): # 지정된 에포크 수만큼 반복
    net.train() # 모델을 학습 모드로 설정
    running_loss = 0.0 # 현재 에포크의 손실 초기화
    for i, data in enumerate(train_loader): # 학습 데이터 로더에서 배치 단위로 데이터 가져오기
      inputs, labels = data  # 입력 데이터와 레이블 분리
      optimizer.zero_grad() # 기울기 초기화
      outputs = net(inputs) # 모델을 통해 예측값 계산
      loss = criterion(outputs, labels)  # 손실 계산
      loss.backward() # 기울기 계산
      optimizer.step() # 파라미터 업데이트
      running_loss += loss.item() # 현재 베치의 손실을 누적

    # 매 에포크마다 평균 학습 손실 계산
    train_loss = running_loss / len(train_loader) # 평균 학습 손실
    train_losses.append(train_loss) # 리스트에 추가

    # 검증 손실 계산
    val_loss = 0.0 # 검증 손실 초기화
    net.eval() # 모델을 평가 모드로 설정
    correct = 0 # 올바른 예측 수 초기화
    total = 0 # 총 에측 수 초기화
    with torch.no_grad(): # 기울기 계산을 하지 않도록 설정
      for inputs, labels in val_loader: # 검증 데이터 로더에서 데이터 가져오기
        outputs = net(inputs) # 모델을 통해 예측값 계산
        _, predicted = torch.max(outputs.data, 1) # 예측값 중 최대값의 인덱스 추출
        total += labels.size(0) # 총 예측 수 증가
        correct += (predicted == labels).sum().item() # 올바른 예측 수 증가
        loss = criterion(outputs, labels) # 검증 손실 계산
        val_loss += loss.item() # 검증 손실 누적


    val_loss /= len(val_loader) # 평균 검증 손실
    val_losses.append(val_loss) # 리스트에 추가

    val_accuracy = 100 * correct / total # 검증 정확도
    val_accuracies.append(val_accuracy) # 리스트에 추가

    # 에포크 결과 출력
    print(f'[{optimizer_name}] Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

  return train_losses, val_losses, val_accuracies  # 학습 손실, 검증 손실, 검증 정확도 반환

In [12]:
train_losses_SGD, val_losses_SGD, val_accuracies_SGD = train_model('SGD', net, train_loader, val_loader, creiterion)

[SGD] Epoch 1, Train Loss: 0.3741, Val Loss: 0.0881, Val Accuracy: 98.50%
[SGD] Epoch 2, Train Loss: 0.1880, Val Loss: 0.1379, Val Accuracy: 96.50%
[SGD] Epoch 3, Train Loss: 0.1398, Val Loss: 0.0829, Val Accuracy: 98.50%
[SGD] Epoch 4, Train Loss: 0.1182, Val Loss: 0.0574, Val Accuracy: 98.50%
[SGD] Epoch 5, Train Loss: 0.0920, Val Loss: 0.0918, Val Accuracy: 98.00%
[SGD] Epoch 6, Train Loss: 0.0910, Val Loss: 0.0451, Val Accuracy: 98.50%
[SGD] Epoch 7, Train Loss: 0.0821, Val Loss: 0.0808, Val Accuracy: 98.50%
[SGD] Epoch 8, Train Loss: 0.0684, Val Loss: 0.0589, Val Accuracy: 98.50%
[SGD] Epoch 9, Train Loss: 0.0782, Val Loss: 0.0416, Val Accuracy: 99.00%
[SGD] Epoch 10, Train Loss: 0.0630, Val Loss: 0.0762, Val Accuracy: 98.50%
[SGD] Epoch 11, Train Loss: 0.0528, Val Loss: 0.0575, Val Accuracy: 98.50%
[SGD] Epoch 12, Train Loss: 0.0901, Val Loss: 0.0811, Val Accuracy: 98.00%
[SGD] Epoch 13, Train Loss: 0.0592, Val Loss: 0.0522, Val Accuracy: 98.50%
[SGD] Epoch 14, Train Loss: 0.0637

In [13]:
# 초기화
net = models.vgg19(pretrained = True) # 사전 훈련된 VGG19 모델을 불러옴

# 모든 파라미터의 requires_grad를 False로 설정하여 학습 중 업데이트되지 않도록 함
for param in net.parameters():
  param.requires_grad = False

# VGG19의 분류기 부분에서 마지막 레이어를 수정
net.classifier[6] = nn.Linear(4096, 2)

# 수정한 마지막 레이어의 requires_grad를 True로 설정하여 학습 중 업데이트되도록 함
net.classifier[6].requires_grad = True

In [14]:
train_losses_Adam, val_losses_Adam, val_accuracies_Adam = train_model('Adam', net, train_loader, val_loader, creiterion)

[Adam] Epoch 1, Train Loss: 0.3606, Val Loss: 0.1691, Val Accuracy: 97.50%
[Adam] Epoch 2, Train Loss: 0.1454, Val Loss: 0.0839, Val Accuracy: 98.50%
[Adam] Epoch 3, Train Loss: 0.1121, Val Loss: 0.1138, Val Accuracy: 97.00%
[Adam] Epoch 4, Train Loss: 0.0945, Val Loss: 0.1158, Val Accuracy: 97.00%
[Adam] Epoch 5, Train Loss: 0.0770, Val Loss: 0.0521, Val Accuracy: 99.00%
[Adam] Epoch 6, Train Loss: 0.0847, Val Loss: 0.1099, Val Accuracy: 97.00%
[Adam] Epoch 7, Train Loss: 0.0633, Val Loss: 0.0579, Val Accuracy: 98.50%
[Adam] Epoch 8, Train Loss: 0.0710, Val Loss: 0.0654, Val Accuracy: 98.50%
[Adam] Epoch 9, Train Loss: 0.0486, Val Loss: 0.1136, Val Accuracy: 96.00%
[Adam] Epoch 10, Train Loss: 0.0439, Val Loss: 0.0535, Val Accuracy: 99.00%
[Adam] Epoch 11, Train Loss: 0.0364, Val Loss: 0.0763, Val Accuracy: 98.00%
[Adam] Epoch 12, Train Loss: 0.0383, Val Loss: 0.0500, Val Accuracy: 98.50%
[Adam] Epoch 13, Train Loss: 0.0273, Val Loss: 0.0618, Val Accuracy: 98.00%
[Adam] Epoch 14, Trai

In [15]:
# 초기화
net = models.vgg19(pretrained = True) # 사전 훈련된 VGG19 모델을 불러옴

# 모든 파라미터의 requires_grad를 False로 설정하여 학습 중 업데이트되지 않도록 함
for param in net.parameters():
  param.requires_grad = False

# VGG19의 분류기 부분에서 마지막 레이어를 수정
net.classifier[6] = nn.Linear(4096, 2)

# 수정한 마지막 레이어의 requires_grad를 True로 설정하여 학습 중 업데이트되도록 함
net.classifier[6].requires_grad = True

In [16]:
train_losses_RAdam, val_losses_RAdam, val_accuracies_RAdam = train_model('RAdam', net, train_loader, val_loader, creiterion)

AttributeError: 'VGG' object has no attribute 'paramters'

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(3, 1, 1)
plt.plot(train_losses_SGD, label='SGD')
plt.plot(train_losses_Adam, label='Adam')
plt.plot(train_losses_RAdam, label='RAdam')
plt.xlabel('Epoch')
plt.ylabel('Train Loss')
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(val_losses_SGD, label='SGD')
plt.plot(val_losses_Adam, label='Adam')
plt.plot(val_losses_RAdam, label='RAdam')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(val_accuracies_SGD, label='SGD')
plt.plot(val_accuracies_Adam, label='Adam')
plt.plot(val_accuracies_RAdam, label='RAdam')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
def load_and_transform_image(image_path, transform):
  # 주어진 경로에서 이미지를 열고 RGB 형식으로 변환
  image = Image.open(image_path).convert('RGB')

  # 변환을 적용하고 차원을 추가하여 배치 형태로 반환
  return transform(image).unsqueeze(0)

In [ ]:
# 클래스 폴더 경로를 딕셔너리로 정의
class_folders = {
    'crack': '/content/drive/MyDrive/Colab Notebooks/ComputerVision/3/test/crack',
    'normal': '/content/drive/MyDrive/Colab Notebooks/ComputerVision/3/test/normal'
}

# 시각화를 위한 플롯 설정
plt.figure(figsize = (20, 8)) # 플롯의 크기를 설정
counter = 1 # 서브플롯 인덱스 초기화

# 각 클래스 이름과 해당 폴더 경로에 대해 반복
for class_name, folder_path in class_folders.items():
  # 폴더 내의 모든 이미지 파일 경로를 가져옴
  image_paths = glob.glob(os.path.join(folder_path, '*'))
  # 첫 5개의 이미지 경로 선택
  selected_paths = image_paths[:5]

  # 선택된 이미지 경로에 대해 반복
  for image_path in selected_paths:
    # 이미지를 로드하고 변환
    image = load_and_transform_image(image_path, transform)

    # 모델을 평가 모드로 설정
    net.eval()
    with torch.no_grad(): # 그래디언트 계산 비활성화
      outputs = net(image) # 모델에 이미지 입력
      _, predicted = torch.max(outputs, 1) # 예측된 클래스 인덱스 추출

    # 예측된 클래스 이름 결정
    predicted_class = 'crack' if predicted.item() == 0 else 'normal'

    # 서브플롯에 이미지와 예측 결과 표시
    plt.subplot(2, 5, counter)
    plt.imshow(Image.open(image_path)) # 원본 이미지를 표시
    plt.title(f'True: {class_name} Predicted: {predicted_class}')
    plt.axis('off') # 축 표시 비활성화
    counter += 1 # 카운터 증가

# 레이아웃 조정 및 플롯 표시
plt.tight_layout() # 서브플롯 간의 간격 조정
plt.show() # 플롯 출력